In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit


from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf, arma_order_select_ic
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.stats.stats import pearsonr

from sklearn.model_selection import TimeSeriesSplit
import time
from datetime import datetime
from tqdm import tqdm


import os
import string

import numpy as np
import pandas as pd
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback


In [10]:
FS = (14, 6)  # figure size
RS = 124  # random state
N_JOBS = 8  # number of parallel threads

# repeated K-folds
N_SPLITS = 10
N_REPEATS = 1

# Optuna
N_TRIALS = 100
MULTIVARIATE = True

# XGBoost
EARLY_STOPPING_ROUNDS = 100

In [13]:
dx = dx = pd.read_csv(r'/home/nkem/Documents/PhD_Research/allN11Oct2022.csv')
dx['incidentdate'] = pd.to_datetime(dx['incidentdate'])
td = dx.copy()
dk = td.groupby([pd.Grouper(key='incidentdate', freq='M')])['estimatedqty'].agg(['sum','size'])
dk = dk.reset_index()
dk.rename(columns={"sum":"estimatedqty", "size":"spillno"}, inplace=True)

df = dk[["incidentdate","spillno"]]
df= df.reset_index()

df["month"] = df["incidentdate"].dt.month
df["year"] = df["incidentdate"].dt.year
dt = df[["incidentdate","month", "year"]]
#dt = dt.set_index("incidentdate")
#target = df[["incidentdate","spillno"]]

dt = dt.set_index("incidentdate")
target = target.set_index("incidentdate")

#data = dt.loc[: "2022-09-30"]
X_train = dt.loc[: "2021-12-31"]
y_train = target.loc[: "2021-12-31"]
#X_valid = dt.loc["2022-01-31" : "2022-06-30"]
#y_valid = target.loc["2022-01-31" : "2022-06-30"]

#X_train = X_train.values
#y_train = y_train.values

In [14]:
X_train.head()

,month,year
incidentdate,,
2005-01-31,1,2005
2005-02-28,2,2005
2005-03-31,3,2005
2005-04-30,4,2005
2005-05-31,5,2005


In [17]:
def objective(
    trial,
    X,
    y,
    random_state=22,
    tscv =TimeSeriesSplit(n_splits=3),
    n_repeats=2,
    n_jobs=1,
    early_stopping_rounds=50,
):
    # XGBoost parameters
    params = {
        "verbosity": 0,  # 0 (silent) - 3 (debug)
        "objective": "reg:squarederror",
        "n_estimators": trial.suggest_int("n_estimator", 1, 10000),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.6),
        "subsample": trial.suggest_loguniform("subsample", 0.4, 0.8),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 10.0),
        "lambda": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "gamma": trial.suggest_loguniform("lambda", 1e-8, 10.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 1000),
        "seed": random_state,
        "n_jobs": n_jobs,
    }

    model = XGBRegressor(**params)
    pruning_callback = XGBoostPruningCallback(trial, "validation_0-rmse")
    rkf = RepeatedKFold(
        n_splits=tscv, n_repeats=n_repeats, random_state=random_state
    )
    X_values = X.values
    y_values = y.values
    y_pred = np.zeros_like(y_values)
    for train_index, test_index in rkf.split(X_values):
        X_A, X_B = X_values[train_index, :], X_values[test_index, :]
        y_A, y_B = y_values[train_index], y_values[test_index]
        model.fit(
            X_A,
            y_A,
            eval_set=[(X_B, y_B)],
            eval_metric="rmse",
            verbose=0,
            callbacks=[pruning_callback],
            early_stopping_rounds=early_stopping_rounds,
        )
        y_pred[test_index] += model.predict(X_B)
    y_pred /= n_repeats
    return np.sqrt(mean_squared_error(y_train, y_pred))

In [20]:
sampler = TPESampler(seed=RS, multivariate=MULTIVARIATE)
study = create_study(direction="minimize", sampler=sampler)
study.optimize(
    lambda trial: objective(
        trial,
        X_train,
        y_train,
        random_state=RS,
        n_repeats=N_REPEATS,
        n_jobs=8,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    ),
    n_trials=N_TRIALS,
    n_jobs=1,
)

# display params
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

[I 2022-11-23 12:29:28,925] A new study created in memory with name: no-name-811b0367-0c4e-4e3a-802a-becb908b32fe
[W 2022-11-23 12:29:28,929] Trial 0 failed because of the following error: ValueError("The number of folds must be of Integral type. TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None) of type <class 'sklearn.model_selection._split.TimeSeriesSplit'> was passed.")
Traceback (most recent call last):
  File "/home/nkem/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_1063080/3407224768.py", line 4, in <lambda>
    lambda trial: objective(
  File "/tmp/ipykernel_1063080/3673883636.py", line 36, in objective
    for train_index, test_index in rkf.split(X_values):
  File "/home/nkem/.local/lib/python3.10/site-packages/sklearn/model_selection/_split.py", line 1414, in split
    cv = self.cv(random_state=rng, shuffle=True, **self.cvargs)
  File "/home/nkem/.local/lib/

ValueError: The number of folds must be of Integral type. TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None) of type <class 'sklearn.model_selection._split.TimeSeriesSplit'> was passed.